In [1]:
import csv
import requests
import os
import sys
from datetime import datetime, timedelta


caminho_commons = os.path.join('..', '..', '..', '..', 'ajna_docs', 'commons')
#print(os.getcwd())
# print(os.listdir(caminho_commons))
sys.path.insert(0, caminho_commons)
from ajna_commons.utils.sanitiza import sanitizar, mongo_sanitizar

with open(os.path.join('..', 'integracao', 'dte.info')) as dte_info:
    linha = dte_info.readline()
DTE_USERNAME = os.environ.get('DTE_USERNAME')
DTE_PASSWORD = os.environ.get('DTE_PASSWORD')
DTE_USERNAME = linha.split(',')[0]
DTE_PASSWORD = linha.split(',')[1]


In [2]:
with open(os.path.join('..', 'integracao', 'recintos.csv'), encoding='utf-8') as csv_in:
    reader = csv.reader(csv_in)
    recintos_list = [row for row in reader]

In [27]:
DTE_TOKEN = 'https://jupapi.org.br/api/sepes/Pesagem/token'
DTE_URL = 'https://jupapi.org.br/api/sepes/PesagemMovimentacao'
# DTE_URL = 'https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem'
def get_token_dte(username=DTE_USERNAME, password=DTE_PASSWORD):
    data = {'username': username, 'password': password, 'grant_type': 'password'}
    r = requests.post(DTE_TOKEN, data=data, verify=False)
    token = r.json().get('access_token')
    return token

In [28]:
data = {'username': DTE_USERNAME, 'password': DTE_PASSWORD, 'grant_type': 'password'}

In [29]:
r = requests.post(DTE_TOKEN, data=data, verify=False)

c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [30]:
r

<Response [200]>

In [31]:
token = r.json().get('access_token')
token

'3GFH-SdXfihPrMjw9leU92IhJLwY0w2eAlyVr-k8yp8I9vT-k45v05Arp02xi_V6z8jB9hd0huLyZJfEXkivzHtr_wvMJ5DNvzip4mXCCeqVJbxiA4xwWNfQtMPUIFfoVDt7RJhWzkCbKZAyAuImBIgbfEcePtjIhmMPzRKhNey23c3K1fiVIJx1ziVheq3i741RCEXSZFue2SDaP48jfvAonuwjb7o_SN0wciRlS-2nOXkY8ipFI4T6Lhu4OcgvRxXSfi_h0sv_liTYwEDlxU3wVSNmfOpJT0fBEV0hExI'

In [32]:

headers = {}
payload = {}
r = requests.post(DTE_URL, headers=headers, params=payload)

In [33]:
r.status_code

404

In [34]:
r.text

'{"Message":"No HTTP resource was found that matches the request URI \'https://jupapi.org.br/api/sepes/PesagemMovimentacao\'."}'

In [35]:
token = get_token_dte()
start = end = datetime.strptime('26/09/2019', '%d/%m/%Y')
payload = {'data_inicio': datetime.strftime(start, '%Y-%m-%d'),
           'data_fim': datetime.strftime(end, '%Y-%m-%d'),
           'cod_recinto': 22}
headers = {'Authorization': 'Bearer ' + token}
r = requests.get(DTE_URL, headers=headers, params=payload)
print(r.url)
print(r.status_code)
lista_pesagens = r.json()


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


https://jupapi.org.br/api/sepes/PesagemMovimentacao?data_inicio=2019-09-26&data_fim=2019-09-26&cod_recinto=22
200


In [36]:
lista_pesagens

{'JUP_WS': {'ID_Processo': '21335',
  'DataHoraProcessamento': '2019-11-01 15:50:18',
  'Status': 'S',
  'Pesagens': {'Lista_Pesagens_Movimentacao': [{'Recinto': 'BRASIL TERMINAL PORTUARIO - DEPOSITARIO',
     'PlacaCavalo': 'BXH5223',
     'ID_BalancaEntrada': '',
     'DataHoraEntrada': '2019-09-26 03:56:57',
     'PlacaCarreta_Entrada': 'DPC8616',
     'VeiculoCarregado_Entrada': 'SIM',
     'CodigoConteiner_Entrada': 'MRKU3250157',
     'PesoBrutoCarga_Entrada': '28060,000',
     'Tara_Entrada': '14510,000',
     'PesoCarregado_Entrada': '42570,000',
     'ID_BalancaSaida': '',
     'DataHoraSaida': '',
     'PlacaCarreta_Saida': '',
     'VeiculoCarregado_Saida': '',
     'CodigoConteiner_Saida': '',
     'PesoBrutoCarga_Saida': '0',
     'Tara_Saida': '0',
     'PesoCarregado_Saida': '0'},
    {'Recinto': 'BRASIL TERMINAL PORTUARIO - DEPOSITARIO',
     'PlacaCavalo': 'BSF4783',
     'ID_BalancaEntrada': '',
     'DataHoraEntrada': '2019-09-26 05:25:42',
     'PlacaCarreta_Entrada

In [17]:
import pickle
with open('lista_pesagens.pkl', 'wb') as file:
    pickle.dump(lista_pesagens, file)

    
#('lista_pesagens.pkl)

In [18]:
with open('lista_pesagens.pkl', 'rb') as file:
    lista_pesagens = pickle.load(file)


In [23]:
pesagens = lista_pesagens['JUP_WS']['Pesagens']['Lista_Pesagens_Movimentacao']
for pesagem in pesagens[:5]:
    print(pesagem)

{'CodigoConteiner_Entrada': 'MRKU3250157', 'PlacaCarreta_Saida': '', 'VeiculoCarregado_Saida': '', 'PesoBrutoCarga_Saida': '0', 'CodigoConteiner_Saida': '', 'PlacaCavalo': 'BXH5223', 'PesoCarregado_Saida': '0', 'ID_BalancaEntrada': '', 'ID_BalancaSaida': '', 'Tara_Entrada': '14510,000', 'VeiculoCarregado_Entrada': 'SIM', 'Recinto': 'BRASIL TERMINAL PORTUARIO - DEPOSITARIO', 'DataHoraEntrada': '2019-09-26 03:56:57', 'Tara_Saida': '0', 'DataHoraSaida': '', 'PesoCarregado_Entrada': '42570,000', 'PlacaCarreta_Entrada': 'DPC8616', 'PesoBrutoCarga_Entrada': '28060,000'}
{'CodigoConteiner_Entrada': 'MSKU8955563', 'PlacaCarreta_Saida': '', 'VeiculoCarregado_Saida': '', 'PesoBrutoCarga_Saida': '0', 'CodigoConteiner_Saida': '', 'PlacaCavalo': 'BSF4783', 'PesoCarregado_Saida': '0', 'ID_BalancaEntrada': '', 'ID_BalancaSaida': '', 'Tara_Entrada': '14520,000', 'VeiculoCarregado_Entrada': 'SIM', 'Recinto': 'BRASIL TERMINAL PORTUARIO - DEPOSITARIO', 'DataHoraEntrada': '2019-09-26 05:25:42', 'Tara_Said

In [28]:
from collections import Counter

cont = Counter()
for pesagem in pesagens:
    camposencontrados = False
    if pesagem['Tara_Entrada'] != '0' or pesagem['Tara_Saida'] != '0':
        cont['tara']+=1
        camposencontrados = True
    if pesagem['CodigoConteiner_Entrada'] or pesagem['CodigoConteiner_Saida']:
        cont['conteiner']+=1
        camposencontrados = True
    if pesagem['VeiculoCarregado_Saida'] == 'SIM':
        cont['saida']+=1
    if pesagem['VeiculoCarregado_Entrada'] == 'SIM':
        cont['entrada']+=1
    if not camposencontrados:
        print(pesagem)
    
print(cont)

{'CodigoConteiner_Entrada': '', 'PlacaCarreta_Saida': 'CUB2498', 'VeiculoCarregado_Saida': 'NÃO', 'PesoBrutoCarga_Saida': '0', 'CodigoConteiner_Saida': '', 'PlacaCavalo': 'BFZ4866', 'PesoCarregado_Saida': '0', 'ID_BalancaEntrada': '', 'ID_BalancaSaida': '', 'Tara_Entrada': '0', 'VeiculoCarregado_Entrada': '', 'Recinto': 'BRASIL TERMINAL PORTUARIO - DEPOSITARIO', 'DataHoraEntrada': '', 'Tara_Saida': '0', 'DataHoraSaida': '2019-09-26 10:11:37', 'PesoCarregado_Entrada': '0', 'PlacaCarreta_Entrada': '', 'PesoBrutoCarga_Entrada': '0'}
{'CodigoConteiner_Entrada': '', 'PlacaCarreta_Saida': 'GQU3460', 'VeiculoCarregado_Saida': 'NÃO', 'PesoBrutoCarga_Saida': '0', 'CodigoConteiner_Saida': '', 'PlacaCavalo': 'HIA8060', 'PesoCarregado_Saida': '0', 'ID_BalancaEntrada': '', 'ID_BalancaSaida': '', 'Tara_Entrada': '0', 'VeiculoCarregado_Entrada': '', 'Recinto': 'BRASIL TERMINAL PORTUARIO - DEPOSITARIO', 'DataHoraEntrada': '', 'Tara_Saida': '0', 'DataHoraSaida': '2019-09-26 11:49:56', 'PesoCarregado_En

In [29]:
print(cont)
print(len(lista_pesagens['JUP_WS']['Pesagens']['Lista_Pesagens_Movimentacao']))

Counter({'tara': 482, 'conteiner': 482, 'saida': 476, 'entrada': 6})
484


In [4]:
def get_pesagens_dte(datainicial, datafinal, recinto, token):
    payload = {'data_inicio': datetime.strftime(datainicial, '%Y-%m-%d'),
               'data_fim': datetime.strftime(datafinal, '%Y-%m-%d'),
               'cod_recinto': recinto}
    headers = {'Authorization': 'Bearer ' + token}
    r = requests.get(DTE_URL, headers=headers, params=payload)
    print(r.url)
    try:
        lista_pesagens = r.json()['JUP_WS']['Pesagens']['Lista_Pesagens']
    except:
        print(r, r.text)
    return lista_pesagens

In [5]:
# Pegar só BTP
recintos_list = [[0], [22], [846]]

In [6]:
from collections import defaultdict
token = get_token_dte()
start = end = datetime.now() - timedelta(days=1)
start = end = datetime.strptime('06/04/2019', '%d/%m/%Y')
pesagens_recintos = defaultdict(list)
for linha in recintos_list[1:]:
    recinto = linha[0]
    pesagens_recinto = get_pesagens_dte(start, end, recinto, token)
    if pesagens_recinto and len(pesagens_recinto) > 0:
        print(recinto, len(pesagens_recinto))
        pesagens_recintos[recinto].extend(pesagens_recinto)

https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem?data_inicio=2019-04-06&data_fim=2019-04-06&cod_recinto=22
22 157
https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem?data_inicio=2019-04-06&data_fim=2019-04-06&cod_recinto=846
846 1202


In [7]:
pesagens_recinto


[{'Recinto': 'BRASIL TERM. PORTUARIO - OPERADOR PORTUA',
  'PlacaCavalo': 'FGA3736',
  'PlacaCarreta': '',
  'DataHoraEntrada': '',
  'ID_BalancaEntrada': '',
  'VeiculoCarregadoEntrada': 'NÃO',
  'CodigoConteinerEntrada': '',
  'PesoEntrada': '0',
  'DataHoraSaida': '',
  'ID_BalancaSaida': '',
  'VeiculoCarregadoSaida': 'NÃO',
  'CodigoConteinerSaida': '',
  'PesoSaida': '0',
  'PesoBrutoCarga': '0,000',
  'MesmaArea_Operador_Recinto': ''},
 {'Recinto': 'BRASIL TERM. PORTUARIO - OPERADOR PORTUA',
  'PlacaCavalo': '',
  'PlacaCarreta': '',
  'DataHoraEntrada': '',
  'ID_BalancaEntrada': '',
  'VeiculoCarregadoEntrada': 'NÃO',
  'CodigoConteinerEntrada': '',
  'PesoEntrada': '0',
  'DataHoraSaida': '',
  'ID_BalancaSaida': '',
  'VeiculoCarregadoSaida': 'NÃO',
  'CodigoConteinerSaida': '',
  'PesoSaida': '0',
  'PesoBrutoCarga': '0,000',
  'MesmaArea_Operador_Recinto': 'SIM'},
 {'Recinto': 'BRASIL TERM. PORTUARIO - OPERADOR PORTUA',
  'PlacaCavalo': '',
  'PlacaCarreta': '',
  'DataHor

In [7]:
for recinto, pesagem_recinto in pesagens_recintos.items():
    print(recinto)
    for pesagem in pesagem_recinto:
        if pesagem.get('CodigoConteinerEntrada') == 'TCLU1454367':
            print(pesagem)
        if pesagem.get('CodigoConteinerSaida') == 'TCLU1454367':
            print(pesagem)

22
{'Recinto': 'Brasil Terminal Portuario', 'PlacaCavalo': 'FGW8436', 'PlacaCarreta': '', 'DataHoraEntrada': '', 'ID_BalancaEntrada': '', 'VeiculoCarregadoEntrada': 'NÃO', 'CodigoConteinerEntrada': '', 'PesoEntrada': '0', 'DataHoraSaida': '2019-04-06 06:03:07', 'ID_BalancaSaida': '', 'VeiculoCarregadoSaida': 'SIM', 'CodigoConteinerSaida': 'TCLU1454367', 'PesoSaida': '38570,000'}
846


In [8]:
import csv
for recinto, pesagem_recinto in pesagens_recintos.items():
    keys = pesagem_recinto[0].keys()
    break
with open('pesagem20190406.csv', 'w', newline='', encoding='utf-8') as csv_out:
    writer = csv.DictWriter(csv_out, fieldnames=['CodRecinto', *keys])
    writer.writeheader()
    for recinto, pesagem_recinto in pesagens_recintos.items():
        for pesagem in pesagem_recinto:
            writer.writerow({'CodRecinto': recinto, **pesagem})


In [6]:
def trata_pesagem_dte(registro):
    new_dict = {}
    for key, value in registro.items():
        key = sanitizar(key, mongo_sanitizar)
        value = sanitizar(value, mongo_sanitizar)
        new_dict[key] = value
    new_dict['datahoraentradaiso'] = datetime.strptime(new_dict['datahoraentrada'], '%Y-%m-%d %H:%M:%S')
    datahorasaida = new_dict['datahorasaida']
    if datahorasaida:
        new_dict['datahorasaidaiso'] = datetime.strptime(new_dict['datahorasaida'], '%Y-%m-%d %H:%M:%S')
    new_dict['pesoentradafloat'] = float(new_dict['pesoentrada'].replace(',', '.'))
    new_dict['pesosaidafloat'] = float(new_dict['pesosaida'].replace(',', '.'))
    new_dict['veiculocarregadosaidabool'] = new_dict['veiculocarregadosaida'] == "sim"
    new_dict['veiculocarregadoentradabool'] = new_dict['veiculocarregadoentrada'] == "sim"
    return(new_dict)

i = 0
for recinto, pesagens in pesagens_recintos.items():
    for pesagem in pesagens:
        if pesagem['CodigoConteinerEntrada'] or pesagem['CodigoConteinerSaida']:
            pesagem_insert_mongo = {'codigo_recinto': recinto}
            pesagem_insert_mongo.update(trata_pesagem_dte(pesagem))
            # print(pesagem_insert_mongo)
            #break
            i += 1
print(i)

1004
